In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Thursday, August 22, 2024
Last Modified: Tuesday, August 12, 2025
Program Purpose: To process the ecoregion file into an array to be used with the IAV calculations.
"""

In [ ]:
#Importing appropriate libraries
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import rasterio
from rasterio import features
# Changing over the font
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block handles the investigation of what the ecoregions file looks in order to be able to convert the file

#Reading the file
ecoregions = geopandas.read_file('wwf_terr_ecos.shp')
#Investigating the file
print(ecoregions)
#Seeing what the map looks like
ecoregions.plot(figsize=(40,20))

In [ ]:
# This block handles the conversion to a numpy array

#Converting the shapefile to the correct projection (Plate-Carree)
ecoregions.to_crs(crs="EPSG:4087") 
ecoregions['ECO_ID'] = ecoregions['ECO_ID'].astype(int)

# Getting the bounds of the shapefile
bounds = ecoregions.total_bounds
print(bounds) #Verifying the bounds: minx, miny, maxx, and maxy
minx = bounds[0]
miny = bounds[1]
maxx = bounds[2]
maxy = bounds[3]

#Calculating the width, height, and aspect ratio to get the correct transformation
width = maxx - minx
height = maxy - miny
aspect_ratio = width / height
print(f"{width}, {height}, {aspect_ratio}") #~~~360, ~~~173.5, ~~~2.07

# Getting desired height based on the width
desired_width = 720
temp_height = int(desired_width / aspect_ratio)
print(temp_height) #347 (will need to stack 13 at the bottom to get 360)

# Getting resolution to make desired size
resolution_x = width / desired_width
resolution_y = height / temp_height

# Creating an affine transformation matrix
affine_transformation = rasterio.transform.from_origin(minx, maxy, resolution_x, resolution_y)

#Making a list of the shape and eco_id as tuples to feed into the rasterize below
shapes_tuples = []
for geometry, eco_id in zip(ecoregions.geometry, ecoregions['ECO_ID']):
    shapes_tuples.append((geometry, eco_id))

# Creating the array of ecoregions on the map with the associated ecoregion ID value
raster = features.rasterize(shapes=shapes_tuples, out_shape=(temp_height, desired_width), fill=0, transform=affine_transformation, dtype='int32')

#Stacking the array so that it is the correct height (need to be 360,720 to work with the other arrays)
raster = np.vstack((np.zeros((13, 720)), raster))

#Sacing the array
np.save('wwf_terr_ecos_360_720.npy', raster)

#Printing what the array looks like now and the dimensions
plt.subplots(figsize=(80,40))
plt.imshow(raster, cmap="Blues")
print(type(raster))
print(np.shape(raster))


In [ ]:
#This block is concerned with verifying what the alignment of the ecoregion and MODIS maps look like

#Verifying the alignment with the MODIS data
temp_array = np.load("MODIS_Map.npy")
temp_array[((temp_array == 0) & (raster != 0)) | ((temp_array != 0) & (raster == 0))] = 99

#Everywhere that is yellow shows where there is a gridcell that is not correctly aligned between the ecoregions and the MODIS maps (can see that it is minimal)
plt.subplots(figsize=(80,40))
plt.imshow(temp_array, interpolation=None)
plt.colorbar()